<div class='alert alert-success'>
    <h5 align="center">VRP</h5>
    <h5 align="center">March 2023</h5>
</div>

* #### importing necessary packages

In [30]:
import pyomo.environ as pyo 
import pandas as pd
import numpy as np

* #### defining the model

In [31]:
model=pyo.ConcreteModel()

* ####  constructing sets and data

In [32]:
model.Iset=pyo.Set(initialize=[1,2,3,4,5,6,7,8,9,10])
model.Vset=pyo.Set(initialize=[0,1,2,3,4,5,6,7,8,9,10])
model.Kset=pyo.Set(initialize=[1,2,3,4])


a={1:2, 2:1, 3:3, 4:3, 5:3, 6:3, 7:2, 8:2, 9:2, 10:3}
b={1:9, 2:7, 3:7, 4:8, 5:10, 6:6, 7:8, 8:9, 9:8, 10:9}
service={0:0, 1:0.25, 2:0.25, 3:0.25, 4:0.25, 5:0.25, 6:0.25, 7:0.25, 8:0.25, 9:0.25, 10:0.25}
demand={1:9, 2:5, 3:6, 4:6, 5:2, 6:9, 7:5, 8:9, 9:2, 10:6}


time= pd.read_excel("D:\\Master of Science\\TA-Integer Programming\\Homework 2\\VRP.xlsx", sheet_name='time',header=0, index_col=0)
time=time.values
time={(i,j):time[i,j] for i in model.Vset for j in model.Vset}


cost= pd.read_excel("D:\\Master of Science\\TA-Integer Programming\\Homework 2\\VRP.xlsx", sheet_name='cost',header=0, index_col=0)
cost=cost.values
cost={(i,j):cost[i,j] for i in model.Vset for j in model.Vset}

* #### defining parameters

In [33]:
model.TravelTime=pyo.Param(model.Vset,model.Vset,initialize=time)

In [34]:
model.cost=pyo.Param(model.Vset,model.Vset,initialize=cost)

In [35]:
model.time_window_1=pyo.Param(model.Iset,initialize=a)

In [36]:
model.time_window_2=pyo.Param(model.Iset,initialize=b)

In [37]:
model.service=pyo.Param(model.Vset,initialize=service)

In [38]:
model.demand=pyo.Param(model.Iset,initialize=demand)

In [39]:
model.c=pyo.Param(initialize=1000)

In [40]:
model.T_Max=pyo.Param(initialize=10)

In [41]:
model.Q=pyo.Param(initialize=21)

* #### defining variables

In [42]:
model.delta=pyo.Var(model.Kset,domain=pyo.Binary)

In [43]:
model.gamma=pyo.Var(model.Vset,model.Vset,model.Kset,domain=pyo.Binary)

In [44]:
model.x=pyo.Var(model.Vset,domain=pyo.NonNegativeReals)

* #### objective function and constraints

In [45]:
def obj(model):
    return model.c*sum(model.delta[k] for k in model.Kset)+sum(model.cost[i,j]*model.gamma[i,j,k] for i in model.Vset for j in model.Vset for k in model.Kset if i!=j)
model.obj = pyo.Objective(rule=obj,sense=pyo.minimize)

In [46]:
def const0_rule(model):
    return model.x[0]==0
model.const0 = pyo.Constraint(rule=const0_rule)

In [47]:
def const1_rule(model,k):
    return sum(model.demand[j]*model.gamma[i,j,k] for i in model.Vset for j in model.Vset  if  j!=0 and i!=j)<=model.Q*model.delta[k]
model.const1 = pyo.Constraint(model.Kset, rule=const1_rule)

In [48]:
def const2_rule(model,k):
    return sum(model.gamma[0,i,k] for i in model.Iset)==model.delta[k]
model.const2 = pyo.Constraint(model.Kset, rule=const2_rule)

In [49]:
def const3_rule(model,k):
    return sum(model.gamma[i,0,k] for i in model.Iset)==model.delta[k]
model.const3 = pyo.Constraint(model.Kset, rule=const3_rule)

In [50]:
def const4_rule(model,k,j):
    return sum(model.gamma[i,j,k] for i in model.Vset if i!=j)==sum(model.gamma[j,i,k] for i in model.Vset if i!=j)
model.const4 = pyo.Constraint(model.Kset,model.Iset, rule=const4_rule)

In [51]:
def const5_rule(model,j):
    return sum(model.gamma[i,j,k] for i in model.Vset for k in model.Kset if i!=j)==1
model.const5 = pyo.Constraint(model.Iset , rule=const5_rule)

In [52]:
def const6_rule(model,i,j,k):
    if i==j or j==0:
        return pyo.Constraint.Skip
    else:
        return model.x[j]>=model.x[i]+(model.service[i]+model.TravelTime[i,j])*model.gamma[i,j,k]-model.T_Max*(1-model.gamma[i,j,k])
model.const6 = pyo.Constraint(model.Vset, model.Vset, model.Kset, rule=const6_rule)

In [53]:
def const7_rule(model,i,k):
    if i==0:
        return pyo.Constraint.Skip
    else:
        return model.x[i]+(model.service[i]+model.TravelTime[i,0])*model.gamma[i,0,k]<=model.T_Max+model.T_Max*(1-model.gamma[i,0,k])
model.const7 = pyo.Constraint(model.Vset, model.Kset, rule=const7_rule)

In [54]:
def const8_rule(model,i):
    return model.x[i]<=model.time_window_2[i]
model.const8 = pyo.Constraint(model.Iset, rule=const8_rule)

In [55]:
def const9_rule(model,i):
    return model.x[i]>=model.time_window_1[i]
model.const9 = pyo.Constraint(model.Iset, rule=const9_rule)

* #### solving the model

In [56]:
opt = pyo.SolverFactory('glpk')
result=opt.solve(model,'glpk')

In [57]:
print(pyo.value(model.obj))

4525.0


In [58]:
model.display()

Model unknown

  Variables:
    delta : Size=4, Index=Kset
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
    gamma : Size=484, Index=gamma_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
          (0, 0, 1) :     0 :  None :     1 : False :  True : Binary
          (0, 0, 2) :     0 :  None :     1 : False :  True : Binary
          (0, 0, 3) :     0 :  None :     1 : False :  True : Binary
          (0, 0, 4) :     0 :  None :     1 : False :  True : Binary
          (0, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
          (0, 1, 2) :     0 :   0.0 :     1 : False : False : Binary
          (0, 1, 3) :     0 :   0.0 :     1 : False : False : Binary
          (0, 1, 4) :     0 :   0.0 :     1 